In [1]:
from pyairtable import Table
import pandas as pd
from PIL import Image
import requests
from sklearn.cluster import KMeans
import numpy as np
# import scipy
# import scipy.misc
# import scipy.cluster
# import binascii
# import imageio
import matplotlib.pyplot as plt
import cv2

In [2]:
api_key = 'keyGkmUD0wzhgeDKj'
base_id = 'app6AHPAHDerGvu0H'
table = 'art_view'
headers = {"Authorization": "Bearer {}".format(api_key)}
url = 'https://api.airtable.com/v0/{}/'.format(base_id)
path = url + table

In [3]:
## retrieve all the data in airtable art_view
params = ()
airtable_records = []
run = True
while run is True:
  response = requests.get(path, params=params, headers=headers)
  airtable_response = response.json()
  airtable_records += (airtable_response['records'])
  if 'offset' in airtable_response:
     run = True
     params = (('offset', airtable_response['offset']),)
  else:
     run = False

In [4]:
## loading the data retrieved from airtable into a pandas dataframe
airtable_rows = []
airtable_index = []
for record in airtable_records:
    airtable_rows.append(record['fields'])
    airtable_index.append(record['id'])
    airtable_dataframe = pd.DataFrame(airtable_rows, index=airtable_index)

In [5]:
# airtable_dataframe[airtable_dataframe['Image']['filename']=='Greta.JPG']

airtable_dataframe['Image'][202]

[{'id': 'attbHuztOFZjirHbR',
  'width': 4032,
  'height': 3024,
  'url': 'https://v5.airtableusercontent.com/v1/16/16/1681552800000/Upew8njomtVVhI_aYs2zTA/CY9qaxy94cL_QHcm_d_xU2dTsyDb9tm2k38jxBMkX6B9E53ofnmDIqiuTH_zW4SFAryptsMjomwy02JTAmpKcwSDMpvM_ndGwSEZ-mXkG_I/X1eulj5wTsR7bx5hiA0tRnAconiCItyRTuZGTEH6WPc',
  'filename': 'Greta.JPG',
  'size': 2614063,
  'type': 'image/jpeg',
  'thumbnails': {'small': {'url': 'https://v5.airtableusercontent.com/v1/16/16/1681552800000/NzWI8t5q8G2ijbHiPWdCfA/KIbDoEQXaNa9z4OUhVkLq_aR37wCYGJ1C6JNKM_t2qC9eREMODYIH2Jwj3OyUMjpW6-wJXXI6aDER2RPwsRDKA/xVcQDX1CEOm0adOYy1WG2GRTmGnPVtPoWLpKjcqoEF0',
    'width': 27,
    'height': 36},
   'large': {'url': 'https://v5.airtableusercontent.com/v1/16/16/1681552800000/5sqxYI4FAl3p0jHWauTvLQ/rJJWPtP_AqRCcvIUvGfdAlt1Z2vFb1wcAkfu0BSSaWWXW0Nq8pK7qHQM_7TK6wn0GuwC63dl_lwXJ4wJK2xDxQ/emH2GIIkMOHTOIt5BG9gTI8cJzs3yZbmL_l2OC9etxQ',
    'width': 512,
    'height': 683},
   'full': {'url': 'https://v5.airtableusercontent.com/v1/16/16

In [6]:
## iterating through all images in the Dataframe and getting the 5 dominant colors
## we then load the RGB color representation of the 5 colors into a dictionary 
## where the key is the index and the values are the RGB color representation

images_colors = {}
image_color_dominance = {}
for ind in airtable_dataframe.index:
    try:
        im_url=airtable_dataframe['Image'][ind][0]['url']
        raw_im =Image.open(requests.get(im_url, stream=True).raw)
        raw_im_array = np.asarray(raw_im)
        r, c = raw_im_array.shape[:2]
        out_r = 500
        new_image = cv2.resize(raw_im_array, (int(out_r*float(c)/r), out_r))
        pixels=new_image.reshape((-1,3))
        k_means= KMeans(n_clusters=5)
        k_means.fit(pixels)
        # print(k_means.cluster_centers_)
        colors = np.asarray(k_means.cluster_centers_, dtype='uint8')
        # print(colors)
        print("getting dominant colors for item " + ind)
        images_colors[ind]=colors
        pixels_colourwise = np.unique(k_means.labels_, return_counts=True)[1]
        percentage = pixels_colourwise/pixels.shape[0]
        image_color_dominance[ind]=percentage
    except:
        raw_im_array_con = np.asarray(raw_im.convert('RGB'))
        r, c = raw_im_array_con.shape[:2]
        out_r = 500
        new_image = cv2.resize(raw_im_array_con, (int(out_r*float(c)/r), out_r))
        pixels=new_image.reshape((-1, 3))
        k_means= KMeans(n_clusters=5)
        k_means.fit(pixels)
        colors = np.asarray(k_means.cluster_centers_, dtype='uint8')
        print("getting dominant colors for item " + ind)
        images_colors[ind]=colors
        pixels_colourwise = np.unique(k_means.labels_, return_counts=True)[1]
        percentage = pixels_colourwise/pixels.shape[0]
        image_color_dominance[ind]=percentage

getting dominant colors for item rec0BCZtgDXCFJHXs
getting dominant colors for item rec0FUvLZ30Z9MLhQ
getting dominant colors for item rec0pii9oPeYeATFY
getting dominant colors for item rec0ymRJmPGlBUloB
getting dominant colors for item rec12j8P3CymIfC6r
getting dominant colors for item rec1uZftEoZnoLo3X
getting dominant colors for item rec2I4d8foEjFqKt9
getting dominant colors for item rec2Pxsp6gbvH8mwG
getting dominant colors for item rec2WCQvS6PxpdlOZ
getting dominant colors for item rec2e3vJQvKOk4TNQ
getting dominant colors for item rec2n7DoXmBS6ifMs
getting dominant colors for item rec35aft7sxc6KfUO
getting dominant colors for item rec3j8E4LqSyLJgjR
getting dominant colors for item rec3tiZTfT4UQZjTv
getting dominant colors for item rec403alLuJU3JIig
getting dominant colors for item rec43i5pfH0XT0aSu
getting dominant colors for item rec48sHungqRYqwgE
getting dominant colors for item rec4Ezg4F3fxhgjxq
getting dominant colors for item rec4NBKeNB4cNPm6Q
getting dominant colors for ite

In [7]:
# we may have to reduce the number of colors in our pallete? or add more
# url = 'https://github.com/chanhojun/dva/blob/main/RGB_Colors.csv?raw=true'
# url_color_pallette = 'https://github.com/codebrainz/color-names/blob/master/output/colors.csv?raw=true'   -----colors without shades
csv = pd.read_csv('Colors_with_shades.csv')


# index=["color", "color_name", "hex", "R", "G", "B"]
#index=["color_name","hex", "rgb", "shades", "R", "G", "B"]
# csv = pd.read_csv(url_color_pallette, names=index, header=None)

#return color name
def recognize_color(R,G,B):
    minimum = 10000
    for i in range(len(csv)):
        d = abs(R- int(csv.loc[i,"R"])) + abs(G- int(csv.loc[i,"G"]))+ abs(B- int(csv.loc[i,"B"]))
        if(d<=minimum):
            minimum = d
            cname = csv.loc[i,"Color Name"]
    return cname

#return color hex
def recognize_color_hex(R,G,B):
    minimum = 10000
    for i in range(len(csv)):
        d = abs(R- int(csv.loc[i,"R"])) + abs(G- int(csv.loc[i,"G"]))+ abs(B- int(csv.loc[i,"B"]))
        if(d<=minimum):
            minimum = d
            hex = csv.loc[i,"Hex Code"]
    return hex


def recognize_color_shade(R,G,B):
    minimum = 10000
    for i in range(len(csv)):
        d = abs(R- int(csv.loc[i,"R"])) + abs(G- int(csv.loc[i,"G"]))+ abs(B- int(csv.loc[i,"B"]))
        if(d<=minimum):
            minimum = d
            hex = csv.loc[i,"Shade"]
    return hex

In [8]:
## Getting color names for the dominnat colors of each image and storing them in a dictionary where the key is the image index and values as color names
image_color_names = {}
for ind, item in images_colors.items():
    image_color_names[ind]=[recognize_color(images_colors[ind][0][0], images_colors[ind][0][1], images_colors[ind][0][2]),
                             recognize_color(images_colors[ind][1][0], images_colors[ind][1][1], images_colors[ind][1][2]),
                             recognize_color(images_colors[ind][2][0], images_colors[ind][2][1], images_colors[ind][2][2]),
                             recognize_color(images_colors[ind][3][0], images_colors[ind][3][1], images_colors[ind][3][2]),
                             recognize_color(images_colors[ind][4][0], images_colors[ind][4][1], images_colors[ind][4][2])]

In [9]:
## getting the hex codes for the dominant colors for each image and storing them in a dictionary where the key is the image index and values as the hex codes
image_color_hex = {}
for ind, item in images_colors.items():
    image_color_hex[ind]=[recognize_color_hex(images_colors[ind][0][0], images_colors[ind][0][1], images_colors[ind][0][2]),
                             recognize_color_hex(images_colors[ind][1][0], images_colors[ind][1][1], images_colors[ind][1][2]),
                             recognize_color_hex(images_colors[ind][2][0], images_colors[ind][2][1], images_colors[ind][2][2]),
                             recognize_color_hex(images_colors[ind][3][0], images_colors[ind][3][1], images_colors[ind][3][2]),
                             recognize_color_hex(images_colors[ind][4][0], images_colors[ind][4][1], images_colors[ind][4][2])]

In [10]:
## getting the hex codes for the dominant colors for each image and storing them in a dictionary where the key is the image index and values as the hex codes
image_color_shade = {}
for ind, item in images_colors.items():
    image_color_shade[ind]=[recognize_color_shade(images_colors[ind][0][0], images_colors[ind][0][1], images_colors[ind][0][2]),
                             recognize_color_shade(images_colors[ind][1][0], images_colors[ind][1][1], images_colors[ind][1][2]),
                             recognize_color_shade(images_colors[ind][2][0], images_colors[ind][2][1], images_colors[ind][2][2]),
                             recognize_color_shade(images_colors[ind][3][0], images_colors[ind][3][1], images_colors[ind][3][2]),
                             recognize_color_shade(images_colors[ind][4][0], images_colors[ind][4][1], images_colors[ind][4][2])]

In [11]:
## convertin the dictionaries for color names, hex codes and color dominance into dataframes.
image_color_names_df=pd.concat({k: pd.Series(v) for k, v in image_color_names.items()}).reset_index(name='color_name')
image_color_dominance_df=pd.concat({k: pd.Series(v) for k, v in image_color_dominance.items()}).reset_index(name='color_dominance')
image_color_hex_df=pd.concat({k: pd.Series(v) for k, v in image_color_hex.items()}).reset_index(name='color_hex')
image_color_shade_df=pd.concat({k: pd.Series(v) for k, v in image_color_shade.items()}).reset_index(name='color_shade')

In [12]:
## renaming the dataframe columns to more descriptive names
image_color_names_df.rename(columns={'level_0':'Image_Index', 'level_1':'df_index'}, inplace=True)
image_color_dominance_df.rename(columns={'level_0':'Image_Index', 'level_1':'df_index'}, inplace=True)
image_color_hex_df.rename(columns={'level_0':'Image_Index', 'level_1':'df_index'}, inplace=True)
image_color_shade_df.rename(columns={'level_0':'Image_Index', 'level_1':'df_index'}, inplace=True)

In [13]:
## Joining the color names dataframe to the color dominance dataframe using the image index and the index for each color for each image
image_colors_with_dominance = pd.merge(image_color_names_df, image_color_dominance_df, on=['Image_Index', 'df_index'])

In [14]:
## joing the resultant dataframe from image colors and color dominance to the hex colors dataframe 
## to get our final dataframe that we will join back to the original airatable dataframe
image_colors_with_hex = pd.merge(image_colors_with_dominance, image_color_hex_df, on=['Image_Index', 'df_index'])

In [15]:
image_colors_full_data = pd.merge(image_colors_with_hex, image_color_shade_df, on=['Image_Index', 'df_index'])

In [16]:
## we then sort our final dataframe from above such that we have the colors for each image sorted in terms of dominance
image_colors_full_data=image_colors_full_data.sort_values(['Image_Index', 'color_dominance'], ascending=[True, False]).reset_index(drop=True)

In [17]:
## we then recreate the index for each image such that the dfindex is ordered in ascending order for each image.
image_colors_full_data['df_index'] = image_colors_full_data.groupby('Image_Index').cumcount()+1

In [18]:
image_colors_full_data.head()

,Image_Index,df_index,color_name,color_dominance,color_hex,color_shade
0,rec0BCZtgDXCFJHXs,1,Dark Brown,0.452945,#5C4033,Brown
1,rec0BCZtgDXCFJHXs,2,Licorice,0.254854,#1B1212,Black
2,rec0BCZtgDXCFJHXs,3,Coffee,0.144145,#6F4E37,Brown
3,rec0BCZtgDXCFJHXs,4,Smoke,0.082652,#848884,Green
4,rec0BCZtgDXCFJHXs,5,Pale Silver,0.065405,#C9C0BB,Orange


In [19]:
## we then convert the final dataframe from a long format to a wide format
image_colors_full_data_wide = image_colors_full_data.pivot_table(index=['Image_Index'], columns='df_index', 
                    values=['color_name', 'color_dominance', 'color_hex', 'color_shade'], aggfunc='first').reset_index()

In [20]:
## we then reset our index and drop the old index
image_colors_full_data_wide=pd.DataFrame(image_colors_full_data_wide.to_records()).reset_index(drop=True)

In [21]:
image_colors_full_data_wide.head()

,index,"('Image_Index', '')","('color_dominance', 1)","('color_dominance', 2)","('color_dominance', 3)","('color_dominance', 4)","('color_dominance', 5)","('color_hex', 1)","('color_hex', 2)","('color_hex', 3)",...,"('color_name', 1)","('color_name', 2)","('color_name', 3)","('color_name', 4)","('color_name', 5)","('color_shade', 1)","('color_shade', 2)","('color_shade', 3)","('color_shade', 4)","('color_shade', 5)"
0,0,rec0BCZtgDXCFJHXs,0.452945,0.254854,0.144145,0.082652,0.065405,#5C4033,#1B1212,#6F4E37,...,Dark Brown,Licorice,Coffee,Smoke,Pale Silver,Brown,Black,Brown,Green,Orange
1,1,rec0FUvLZ30Z9MLhQ,0.289194,0.222516,0.215958,0.179149,0.093182,#28282B,#71797E,#51414F,...,Matte Black,Steel Gray,Quartz,Smoke,Pale Silver,Black,Gray,Purple,Green,Orange
2,2,rec0pii9oPeYeATFY,0.318602,0.252782,0.227511,0.114508,0.086598,#6F4E37,#8A9A5B,#483C32,...,Coffee,Moss Green,Taupe,Steel Gray,Pear,Brown,Green,Brown,Gray,Green
3,3,rec0ymRJmPGlBUloB,0.533568,0.247752,0.118848,0.061200,0.038632,#1B1212,#28282B,#51414F,...,Licorice,Matte Black,Quartz,Granite,Rose Gold,Black,Black,Purple,Gray,Pink
4,4,rec12j8P3CymIfC6r,0.355750,0.199776,0.192212,0.138237,0.114026,#1B1212,#625D5D,#E5E4E2,...,Licorice,Carbon Gray,Platinum,Dark Brown,Smoke,Black,Gray,Gray,Brown,Green


In [22]:
## we then rename the columns to have descriptive names.
image_colors_full_data_wide.rename(columns={"('Image_Index', '')": "Image_Index", "('color_name', 1)":"Dominant_color_1", "('color_name', 2)":"Dominant_color_2",
                                                 "('color_name', 3)":"Dominant_color_3", "('color_name', 4)":"Dominant_color_4", "('color_name', 5)":"Dominant_color_5",
                                                 "('color_dominance', 1)":"color_1_dominance", "('color_dominance', 2)":"color_2_dominance", 
                                                 "('color_dominance', 3)":"color_3_dominance", "('color_dominance', 4)":"color_4_dominance",
                                                 "('color_dominance', 5)":"color_5_dominance", "('color_hex', 1)":"color_hex_1", "('color_hex', 2)":"color_hex_2",
                                                 "('color_hex', 3)":"color_hex_3", "('color_hex', 4)":"color_hex_4", "('color_hex', 5)":"color_hex_5",
                                                 "('color_shade', 1)":"color_shade_1", "('color_shade', 2)":"color_shade_2", "('color_shade', 3)":"color_shade_3",
                                                 "('color_shade', 4)":"color_shade_4", "('color_shade', 5)":"color_shade_5"}, inplace = True)

In [23]:
## we finally then join our wide dataframe containing the image colors, dominance and hex codes to our original airtable dataframe.
new_modified_dataframe = pd.merge(airtable_dataframe, image_colors_full_data_wide, left_index=True, right_on='Image_Index')

In [24]:
## we then export the final dataframe to csv which we will use for analysis and visualization
new_modified_dataframe.to_csv('data_with_color_names_ranked.csv')